# Import

In [1]:
import os
import sys
import json
import pandas as pd
import numpy as np
import torch
import pickle
import pathSetting
import Parameter as P

sys.path.append("../")
sys.path.append("../../")
sys.path.append("../../..")

from KETIToolDL.CLUSTTool.common import p1_integratedDataSaving as p1
import getIntegrationData as gID

c:\Users\korea\anaconda3\envs\torch\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# 1. Data Split&Transform&Integration
## 1-1. 3D Array Data Read

In [2]:
# 1. UCI HAR Data Read
folderAddress = './data/'
dataset_train_x = pickle.load(open(folderAddress + 'x_train.pkl', 'rb'))
dataset_train_y = pickle.load(open(folderAddress + 'y_train.pkl', 'rb'))
dataset_test_x = pickle.load(open(folderAddress + 'x_test.pkl', 'rb'))
dataset_test_y = pickle.load(open(folderAddress + 'y_test.pkl', 'rb'))

In [3]:
# 1-1. train shape 확인
dataset_train_x.shape, dataset_train_y.shape

((7352, 9, 128), (7352,))

In [4]:
# 1-2. test shape 확인
dataset_test_x.shape, dataset_test_y.shape

((2947, 9, 128), (2947,))

## 1-2. Basic Setting

In [5]:
# 2. Basic Setting
dataModeList = ['trainX', 'trainy', 'testX', 'testy']
transformModeList = ['splitAndTransform', 'onlyTransform']

# 2-1. 변형 및 통합을 원하는 데이터 모드 선택
dataModeX = dataModeList[0]
dataModey = dataModeList[1]

# 2-2. 분할 및 변형 방법 모드 선택 : train/test 둘 다 X데이터일때만 선택
if (dataModeX == "trainX") | (dataModeX == "testX"):
    transformMode = transformModeList[0]

# 2-3. set parameter
trainStartTime = "2022-01-01"
testStartTime = "2022-02-01"
integration_method = "ML" #ML/meta
window_size = 128 # ML 통합 시 필요한것으로 rnn ae 학습 시 활용되는 window size. 데이터 split&transform 했을때 하나의 데이터 길이(데이터셋 길이가 아닌)
sliding_size = 128 # rnn

intParam = P.setParameter(integration_method, window_size, sliding_size)

cuda is available.


In [6]:
# 2-4. 선택한 모드에 따라 parameter 설정
if dataModeX == 'trainX':
    originalDataset_X = dataset_train_x.copy()
    originalDataset_y = dataset_train_y.copy()
    startTime = trainStartTime
    if transformMode == 'splitAndTransform':
        splitNum = 3
        transformFreqList = ['1S', '2S', '4S']
    elif transformMode == 'onlyTransform':
        splitNum = 1
        transformFreqList = ['4S']
    
elif dataModeX == 'testX':
    originalDataset_X = dataset_test_x.copy()
    originalDataset_y = dataset_test_y.copy()
    startTime = testStartTime
    if transformMode == 'splitAndTransform':
        splitNum = 3
        transformFreqList = ['1S', '2S', '4S']
    elif transformMode == 'onlyTransform':
        splitNum = 1
        transformFreqList = ['4S']

## 1-3. Get IntegrationData & window

In [7]:
# 3. 분할 및 변형, 통합 데이터 생성
# 3-1. 모드에 따라 설정한 기본 parameter와 P.parameter를 입력
gid = gID.getIntegrationData(originalDataset_X, startTime, splitNum, transformFreqList, **intParam)
intDataSet_X, DataSet_y, window = gid.getIntegrationDataByMethod(mode = dataModeX, data_y = originalDataset_y)

>>>> class  0  , length :  1226
====== start trans3NP to DF ======
df_x length :  156928
====== start split & transform ======


UnboundLocalError: local variable 'split_data' referenced before assignment

In [33]:
# 3-2. 통합 데이터
intDataSet_X

,concat_emb1,concat_emb2,concat_emb3,concat_emb4,concat_emb5,concat_emb6,concat_emb7,concat_emb8,concat_emb9,concat_emb10,...,concat_emb23,concat_emb24,concat_emb25,concat_emb26,concat_emb27,concat_emb28,concat_emb29,concat_emb30,concat_emb31,concat_emb32
2022-01-01 00:00:00,0.025712,-0.148021,-0.018816,0.024458,-0.097670,0.080558,-0.028627,-0.158008,-0.085572,0.091588,...,0.145292,0.137852,-0.137662,0.051408,-0.203874,-0.128454,0.022482,-0.007411,-0.011939,-0.121673
2022-01-01 00:00:04,0.007098,-0.152861,-0.026520,0.009311,-0.083828,0.079895,-0.022634,-0.146849,-0.089867,0.103487,...,0.147611,0.128161,-0.125431,0.045352,-0.203235,-0.132101,0.029660,-0.002992,-0.011133,-0.141619
2022-01-01 00:00:08,0.017475,-0.154284,-0.019217,0.015391,-0.090105,0.082844,-0.021357,-0.144400,-0.098256,0.101111,...,0.145854,0.127975,-0.125641,0.052064,-0.206044,-0.135248,0.028694,0.000143,-0.005179,-0.135766
2022-01-01 00:00:12,0.049457,-0.141920,-0.012297,0.018949,-0.143499,0.114135,-0.031460,-0.168569,-0.082355,0.092348,...,0.134737,0.127335,-0.141193,0.070764,-0.207523,-0.130398,0.006963,0.003658,0.016051,-0.120061
2022-01-01 00:00:16,0.051669,-0.176772,-0.017089,0.042627,-0.123923,0.076749,-0.021107,-0.159368,-0.100059,0.099691,...,0.168474,0.131372,-0.143223,0.064790,-0.221059,-0.145697,0.023680,0.010304,-0.002695,-0.148706
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-01-01 08:09:48,0.146284,0.109511,-0.080031,0.084099,-0.008976,0.103823,-0.129748,0.015084,-0.037604,-0.003279,...,-0.098881,0.141546,0.153031,-0.240717,0.192737,-0.093447,0.082651,-0.182699,-0.004390,-0.001884
2022-01-01 08:09:52,0.146171,0.109479,-0.079950,0.084039,-0.009100,0.103816,-0.129462,0.015117,-0.037604,-0.003293,...,-0.098901,0.141518,0.153106,-0.240787,0.192635,-0.093523,0.082501,-0.182471,-0.004494,-0.002010
2022-01-01 08:09:56,0.146182,0.109508,-0.079733,0.084046,-0.009160,0.103864,-0.129720,0.014982,-0.037428,-0.003151,...,-0.098974,0.141516,0.153143,-0.240961,0.192796,-0.093472,0.082487,-0.182522,-0.004649,-0.001921
2022-01-01 08:10:00,0.146151,0.109582,-0.079514,0.084064,-0.009254,0.103868,-0.129848,0.014972,-0.037175,-0.002936,...,-0.099082,0.141569,0.153253,-0.241304,0.192813,-0.093516,0.082474,-0.182593,-0.004618,-0.001859


In [34]:
DataSet_y

,value
2022-01-01 00:00:00,0.0
2022-01-01 00:00:01,0.0
2022-01-01 00:00:02,0.0
2022-01-01 00:00:03,0.0
2022-01-01 00:00:04,0.0
...,...
2022-01-01 02:02:27,5.0
2022-01-01 02:02:28,5.0
2022-01-01 02:02:29,5.0
2022-01-01 02:02:30,5.0


In [35]:
# timeElapsed 설정
timeElapsed = gid.timeElapsed

## 1-4. Data Save & Json Save

In [36]:
# 4. Data&Json Save
# 4-1. Json Info 설정
datasetName = 'ActionPatternData'
processParam = intParam["processParam"]
dataInfo = {}
integration_freq_sec = intParam["integration_freq_sec"]
cleanParam = intParam["cleanParam"]
DataSaveMode = intParam["DataSaveMode"]

In [37]:
for name in [dataModeX, dataModey]:
    dataMode = name # dataModey
    if name[-1] == "X":
        data = intDataSet_X.copy()
    else:
        data = DataSet_y.copy()
        
    dataName = integration_method+"_"+dataMode+'_'+datasetName+'_Split'+str(splitNum)
    endTime = str(data.index[-1])
    dataInfo = {
        "windows" : 1,
        "dataSize" : str(data.size),
        "timeElapsed" : timeElapsed
    }
    
    print(dataName)
    #print(data.info())
    
    # 4-3. Data Save
    fileName = p1.saveCSVData(pathSetting.dataFolderPath, dataName, data)

    # 4-4. Json Save
    p1.saveJsonMeta(pathSetting.DataMetaPath, dataName, processParam, dataInfo,integration_freq_sec, 
                    startTime, endTime, cleanParam, DataSaveMode)

ML_trainX_ActionPatternData_Split3
ML_trainy_ActionPatternData_Split3
